In [2]:
import numpy as np
np.random.seed(1337)
import json, re, nltk, string
from nltk.corpus import wordnet
from gensim.models import Word2Vec
from keras.preprocessing import sequence
from keras.models import Model
from keras.layers import Dense, Dropout, Embedding, LSTM, Input, merge
from keras.optimizers import RMSprop
from keras.utils import np_utils
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
import pandas as pd

In [5]:
all_bugs_json = 'D:\\BugTriage\\Chrome\\all_data.json'
closed_bugs_json = 'D:\\BugTriage\\Chrome\\classifier_data_0.json'

In [6]:
with open(all_bugs_json) as data_file:
    data = json.load(data_file, strict=False)


In [7]:
data_id= list()
data_issueId = list()
data_issueTitle = list()
data_reportedTime = list()
data_owner = list()
data_description = list()

data_des = []

for i in range(len(data)):
    data_id.append(data[i]['id'])
    data_issueId.append(data[i]['issue_id'])
    data_issueTitle.append(data[i]['issue_title'])
    data_reportedTime.append(data[i]['reported_time'])
    data_owner.append(data[i]['owner'])
    data_des.append(data[i]['description'])

for item in data_des:
    current_desc = item.replace('\r', ' ')
    current_desc = current_desc.replace('\n', ' ')    
    current_desc = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', current_desc)
    current_desc = current_desc[:start_loc]  
    current_desc = re.sub(r'(\w+)0x\w+', '', current_desc)
    current_desc = current_desc.lower()
    current_desc_tokens = nltk.word_tokenize(current_desc)
    current_desc_filter = [word.strip(string.punctuation) for word in current_desc_tokens]

    data_description.append(current_desc)

In [8]:
for item in data:
    current_title = item['issue_title'].replace('\r', ' ')
    current_desc = item['description'].replace('\r', ' ')
    current_desc = current_desc.replace('\n', ' ')

    current_desc = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', current_desc)

    start_loc = current_desc.find("Stack trace:")
    current_desc = current_desc[:start_loc]
    

    current_desc = re.sub(r'(\w+)0x\w+', '', current_desc)
    current_title= re.sub(r'(\w+)0x\w+', '', current_title)

    current_desc = current_desc.lower()
    current_title = current_title.lower()
    #print(current_title)

    current_desc = current_title + current_desc
    #print(current_desc)
    '''current_desc_tokens = nltk.word_tokenize(current_desc)
    current_title_tokens = nltk.word_tokenize(current_title)
    print(current_desc_tokens)
    current_desc_filter = [word.strip(string.punctuation) for word in current_desc_tokens]
    current_title_filter = [word.strip(string.punctuation) for word in current_title_tokens]      

    current_data = current_title_filter + current_desc_filter
    current_data = filter(None, current_data)'''

    data_description.append(current_desc)

In [ ]:
data_description

In [12]:
data_owner

['',
 '',
 'amit@chromium.org',
 '',
 '',
 'venkataramana@chromium.org',
 '',
 '',
 '',
 'mal.chro...@gmail.com',
 '',
 '',
 '',
 '',
 '',
 '',
 'aa@chromium.org',
 'cbentzel@chromium.org',
 '',
 'eroman@chromium.org',
 '',
 '',
 'jon@chromium.org',
 'pfeldman@chromium.org',
 '',
 '',
 '',
 '',
 '',
 'eroman@chromium.org',
 '',
 '',
 'jon@chromium.org',
 'pkasting@chromium.org',
 '',
 'jon@chromium.org',
 '',
 'kuchhal@chromium.org',
 '',
 'jon@chromium.org',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'brettw@chromium.org',
 '',
 'jon@chromium.org',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'amit@chromium.org',
 'ben@chromium.org',
 '',
 'thestig@chromium.org',
 '',
 'sky@chromium.org',
 'sky@chromium.org',
 '',
 'mal.chro...@gmail.com',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'darin@chromium.org',
 'ben@chromium.org',
 'amit@chromium.org',
 'thestig@chromium.org',
 '',
 '',
 '',
 'thestig@chromium.org',
 '',
 'venkataramana@chromium.org',
 'sky@chromium.org',
 '',
 'dcheng@chromium.org',
 ''

In [ ]:
issue_data = pd.DataFrame(data = {'issue_title': issue_title, 'description': description})
r